Lab Statement 7

(1) Extend the Apriori Algorithm discussed in the class supporting Transaction Reduction
approach to improve the time complexity issue as a result of the repeated scans limitation of
Apriori. You may compare this extended version with the earlier implementations in FP
Growth over the same benchmark dataset.

In [ ]:
import time
from collections import defaultdict
from itertools import combinations

# Function to load the mushroom dataset
def load_mushroom_data(file_path):
    transactions = []
    with open(file_path, 'r') as file:
        for line in file:
            # Split each line into items (space-separated)
            transaction = line.strip().split()
            transactions.append(transaction)
    return transactions

# Helper function to calculate support
def calculate_support(itemset, transactions):
    return sum(1 for transaction in transactions if set(itemset).issubset(set(transaction)))

# Apriori Algorithm with Transaction Reduction
def apriori_transaction_reduction(transactions, min_support):
    items = {item for transaction in transactions for item in transaction}
    candidates = [{item} for item in items]
    frequent_itemsets = []

    while candidates:
        candidate_support = {frozenset(candidate): calculate_support(candidate, transactions) for candidate in candidates}
        frequent_candidates = {itemset: support for itemset, support in candidate_support.items() if support >= min_support}

        if not frequent_candidates:
            break

        transactions = [[item for item in transaction if {item}.issubset(set(frequent_candidates))] for transaction in transactions]
        frequent_itemsets.extend(frequent_candidates)

        candidate_items = set().union(*frequent_candidates)
        next_candidate_size = len(list(frequent_candidates)[0]) + 1
        candidates = [set(comb) for comb in combinations(candidate_items, next_candidate_size) if len(set(comb)) == next_candidate_size]

    return frequent_itemsets

# FP-Growth Algorithm
class TreeNode:
    def __init__(self, item, count=0):
        self.item = item
        self.count = count
        self.children = {}
        self.parent = None
        self.link = None

def insert_tree(items, node, header_table):
    first_item = items[0]
    if first_item in node.children:
        node.children[first_item].count += 1
    else:
        new_node = TreeNode(first_item, 1)
        new_node.parent = node
        node.children[first_item] = new_node

        if first_item in header_table:
            last_node = header_table[first_item][1]
            while last_node.link is not None:
                last_node = last_node.link
            last_node.link = new_node
        else:
            header_table[first_item] = (1, new_node)

    if len(items) > 1:
        insert_tree(items[1:], node.children[first_item], header_table)

def fp_growth(transactions, min_support):
    item_freq = defaultdict(int)
    for transaction in transactions:
        for item in transaction:
            item_freq[item] += 1

    item_freq = {k: v for k, v in item_freq.items() if v >= min_support}

    root = TreeNode(None)
    header_table = {}

    for transaction in transactions:
        sorted_items = [item for item in sorted(transaction, key=lambda i: item_freq.get(i, 0), reverse=True) if item in item_freq]
        if sorted_items:
            insert_tree(sorted_items, root, header_table)

    return item_freq

# Function to compare running times
def compare_algorithms(transactions, min_support_ratio):
    total_transactions = len(transactions)
    min_support = int(min_support_ratio * total_transactions)

    apriori_start_time = time.time()
    apriori_result = apriori_transaction_reduction(transactions, min_support)
    apriori_time = time.time() - apriori_start_time

    fp_growth_start_time = time.time()
    fp_growth_result = fp_growth(transactions, min_support)
    fp_growth_time = time.time() - fp_growth_start_time

    return apriori_time, fp_growth_time

# Sample execution using the mushroom.dat dataset
file_path = 'mushroom.dat'  # Path to mushroom.dat
transactions = load_mushroom_data(file_path)
min_support_ratio = 0.6  # Set the minimum support ratio (30%)

# Compare running times
apriori_time, fp_growth_time = compare_algorithms(transactions, min_support_ratio)

# Print results only once
print("RUNNING TIME OF ALGORITHMS FOR TRANSACTIONS OF mushroom.dat FIMI DATASET\n")
print(f"Apriori Time: {apriori_time:.4f} seconds")
print(f"FP-Growth Time: {fp_growth_time:.4f} seconds\n")
print("It is noted that FP-Growth takes less time than Apriori for finding frequent itemset in same transactions")

RUNNING TIME OF ALGORITHMS FOR TRANSACTIONS OF mushroom.dat FIMI DATASET

Apriori Time: 1.0472 seconds
FP-Growth Time: 0.0578 seconds

It is noted that FP-Growth takes less time than Apriori for finding frequent itemset in same transactions


(2) Test drive any one implementation in FP growth or Transaction Reduction adopting a Vertical Transaction Database format.


In [ ]:
import time
from collections import defaultdict
from itertools import combinations

# Function to load the mushroom dataset
def load_mushroom_data(file_path):
    transactions = []
    with open(file_path, 'r') as file:
        for line in file:
            # Split each line into items (space-separated)
            transaction = line.strip().split()
            transactions.append(transaction)
    return transactions

# Convert horizontal transactions to vertical transaction database (VTD)
def convert_to_vertical_db(transactions):
    vertical_db = defaultdict(set)
    for tid, transaction in enumerate(transactions):
        for item in transaction:
            vertical_db[item].add(tid)
    return vertical_db

# Helper function to get the intersection of sets
def get_intersection(itemset, vertical_db):
    # Check if all items in the itemset are in the vertical_db
    if all(item in vertical_db for item in itemset):
        item_tids = [vertical_db[item] for item in itemset]
        return set.intersection(*item_tids) if item_tids else set()
    else:
        return set()

# Transaction Reduction using Vertical Transaction Database format
def apriori_vtd(transactions, min_support_count):
    vertical_db = convert_to_vertical_db(transactions)
    frequent_itemsets = []

    # Initialize single itemsets (1-itemsets)
    candidates = [{item} for item in vertical_db]

    while candidates:
        # Calculate support and form frequent candidates
        candidate_support = {frozenset(candidate): len(get_intersection(candidate, vertical_db)) for candidate in candidates}
        frequent_candidates = {itemset: support for itemset, support in candidate_support.items() if support >= min_support_count}

        if not frequent_candidates:
            break

        # Reduce vertical_db based on frequent candidates
        reduced_db = {item: tids for item, tids in vertical_db.items() if {item}.issubset(set(frequent_candidates))}
        vertical_db = reduced_db
        frequent_itemsets.extend(frequent_candidates.items())  # Store as tuples (itemset, support)

        # Generate next candidate size (k-itemsets)
        candidate_items = set().union(*frequent_candidates)
        next_candidate_size = len(list(frequent_candidates)[0]) + 1
        candidates = [set(comb) for comb in combinations(candidate_items, next_candidate_size) if len(set(comb)) == next_candidate_size]

    return frequent_itemsets

# Function to calculate minimum support count
def calculate_min_support_count(transactions, min_support_ratio):
    total_transactions = len(transactions)
    return int(min_support_ratio * total_transactions)

# Sample execution using the mushroom.dat dataset
file_path = 'mushroom.dat'  # Path to mushroom.dat
transactions = load_mushroom_data(file_path)
min_support_ratio = 0.1 # Set the minimum support ratio

# Calculate minimum support count
min_support_count = calculate_min_support_count(transactions, min_support_ratio)

# Run the Apriori algorithm using VTD format
start_time = time.time()
frequent_itemsets_vtd = apriori_vtd(transactions, min_support_count)
end_time = time.time()

# Print results
print("Frequent Itemsets using Vertical Transaction Database (VTD) format with min support count:\n")

# Counter for number of frequent itemsets
num_frequent_itemsets = 0

for itemset, support in frequent_itemsets_vtd:  # Now expect (itemset, support)
    print(f"Itemset: {set(itemset)}, Total occurence: {support}")
    num_frequent_itemsets += 1

# Print number of frequent itemsets
print(f"\nTotal Number of Frequent Itemsets: {num_frequent_itemsets}")
print(f"Execution Time: {end_time - start_time:.4f} seconds")

Frequent Itemsets using Vertical Transaction Database (VTD) format with min support count:

Itemset: {'1'}, Total occurence: 3916
Itemset: {'3'}, Total occurence: 3656
Itemset: {'9'}, Total occurence: 2556
Itemset: {'13'}, Total occurence: 2284
Itemset: {'23'}, Total occurence: 3376
Itemset: {'34'}, Total occurence: 7914
Itemset: {'36'}, Total occurence: 6812
Itemset: {'38'}, Total occurence: 2512
Itemset: {'52'}, Total occurence: 3516
Itemset: {'54'}, Total occurence: 1120
Itemset: {'59'}, Total occurence: 5176
Itemset: {'63'}, Total occurence: 4936
Itemset: {'67'}, Total occurence: 4464
Itemset: {'76'}, Total occurence: 4384
Itemset: {'85'}, Total occurence: 8124
Itemset: {'86'}, Total occurence: 7924
Itemset: {'90'}, Total occurence: 7488
Itemset: {'93'}, Total occurence: 3968
Itemset: {'98'}, Total occurence: 1872
Itemset: {'107'}, Total occurence: 1248
Itemset: {'2'}, Total occurence: 4208
Itemset: {'14'}, Total occurence: 1072
Itemset: {'39'}, Total occurence: 5612
Itemset: {'99'

(3) Using a vertical transaction database notation, generate the FI’s following the intersection
approach (basic ECLAT) discussed in the class. Use benchmark datasets in FIMI.

In [ ]:
import time
from collections import defaultdict

# Function to load the dataset
def load_dataset(file_path):
    transactions = []
    with open(file_path, 'r') as file:
        for line in file:
            transaction = line.strip().split()
            transactions.append(transaction)
    return transactions

# Function to convert transactions to vertical format
def convert_to_vertical(transactions):
    vertical_db = defaultdict(list)
    for idx, transaction in enumerate(transactions):
        for item in transaction:
            vertical_db[item].append(idx)  # Append transaction index for each item
    return vertical_db

# ECLAT implementation using vertical database format
def eclat(vertical_db, min_support_count):
    # Initialize frequent itemsets
    frequent_itemsets = {}

    # Convert vertical database into a list of items
    items = list(vertical_db.keys())

    # Helper function for recursive ECLAT
    def eclat_recursive(prefix, items, vertical_db):
        for i in range(len(items)):
            item = items[i]
            new_prefix = prefix + [item]
            transactions = vertical_db[item]

            # Check the support
            if len(transactions) >= min_support_count:
                frequent_itemsets[frozenset(new_prefix)] = len(transactions)

                # Continue recursively
                eclat_recursive(new_prefix, items[i+1:], vertical_db)

    # Start the recursive ECLAT
    eclat_recursive([], items, vertical_db)

    return frequent_itemsets

# Function to compare running times and execute ECLAT
def compare_eclat(transactions, min_support_ratio):
    total_transactions = len(transactions)
    min_support_count = int(min_support_ratio * total_transactions)  # Calculate the minimum support count

    vertical_db = convert_to_vertical(transactions)

    start_time = time.time()
    frequent_itemsets = eclat(vertical_db, min_support_count)
    execution_time = time.time() - start_time

    return frequent_itemsets, execution_time

# Sample execution using a benchmark dataset from FIMI
file_path = 'mushroom.dat'
transactions = load_dataset(file_path)
min_support_ratio = 0.6  # Set minimum support as a ratio

# Run the ECLAT algorithm
frequent_itemsets, execution_time = compare_eclat(transactions, min_support_ratio)

# Print results in a structured format
print("Frequent Itemsets (count: {}):".format(len(frequent_itemsets)))
for i, itemset in enumerate(frequent_itemsets.keys(), start=1):
    print(f"{i}. {sorted(list(itemset))}: {frequent_itemsets[itemset]}")  # Print itemset and its support

print(f"\nExecution Time: {execution_time:.4f} seconds")


Frequent Itemsets (count: 255):
1. ['34']: 7914
2. ['34', '36']: 6812
3. ['34', '36', '59']: 5176
4. ['34', '36', '59', '63']: 4936
5. ['34', '36', '59', '63', '85']: 8124
6. ['34', '36', '59', '63', '85', '86']: 7924
7. ['34', '36', '59', '63', '85', '86', '90']: 7488
8. ['34', '36', '39', '59', '63', '85', '86', '90']: 5612
9. ['34', '36', '39', '59', '63', '85', '86']: 5612
10. ['34', '36', '59', '63', '85', '90']: 7488
11. ['34', '36', '39', '59', '63', '85', '90']: 5612
12. ['34', '36', '39', '59', '63', '85']: 5612
13. ['34', '36', '59', '63', '86']: 7924
14. ['34', '36', '59', '63', '86', '90']: 7488
15. ['34', '36', '39', '59', '63', '86', '90']: 5612
16. ['34', '36', '39', '59', '63', '86']: 5612
17. ['34', '36', '59', '63', '90']: 7488
18. ['34', '36', '39', '59', '63', '90']: 5612
19. ['34', '36', '39', '59', '63']: 5612
20. ['34', '36', '59', '85']: 8124
21. ['34', '36', '59', '85', '86']: 7924
22. ['34', '36', '59', '85', '86', '90']: 7488
23. ['34', '36', '39', '59', '85'

(4) Extend the basic Apriori algorithm to generate Frequent Patterns which differentiate ab
from ba (ordered patterns generation).


In [ ]:
from collections import defaultdict

def get_frequent_itemsets_apriori(dataset, min_support):
    # Get 1-itemsets
    item_count = defaultdict(int)
    for transaction in dataset:
        for item in transaction:
            item_count[item] += 1

    frequent_items = {frozenset([item]) for item, count in item_count.items() if count >= min_support}
    frequent_itemsets = frequent_items.copy()

    # Generate higher-order itemsets with ordered patterns (ab ≠ ba)
    k = 2
    ordered_patterns = set()
    while frequent_items:
        candidate_itemsets = {i.union(j) for i in frequent_items for j in frequent_items if len(i.union(j)) == k}
        item_count = defaultdict(int)

        for transaction in dataset:
            for itemset in candidate_itemsets:
                if itemset.issubset(transaction):
                    item_count[itemset] += 1
                    # Ordered pattern generation (ab ≠ ba)
                    if tuple(sorted(itemset)) not in ordered_patterns:
                        ordered_patterns.add(tuple(itemset))

        frequent_items = {itemset for itemset, count in item_count.items() if count >= min_support}
        frequent_itemsets.update(frequent_items)
        k += 1

    return frequent_itemsets, ordered_patterns

# Run the Apriori algorithm with ordered patterns
apriori_frequent_itemsets, apriori_ordered_patterns = get_frequent_itemsets_apriori(dataset, min_support)

print(f"Total number of frequent items (Apriori): {len(apriori_frequent_itemsets)}")
print("Frequent items:", apriori_frequent_itemsets)
print(f"Total number of ordered patterns: {len(apriori_ordered_patterns)}")
print("Ordered patterns:", apriori_ordered_patterns)


Total number of frequent items (Apriori): 51
Frequent items: {frozenset({'85', '90', '86'}), frozenset({'85', '39', '86'}), frozenset({'34', '85', '39', '86'}), frozenset({'85'}), frozenset({'39'}), frozenset({'36'}), frozenset({'34'}), frozenset({'36', '85', '34', '86'}), frozenset({'59', '85', '86'}), frozenset({'85', '90'}), frozenset({'36', '34', '86'}), frozenset({'36', '90', '86'}), frozenset({'90', '34', '86'}), frozenset({'90', '34'}), frozenset({'59', '34', '86'}), frozenset({'34', '85', '39'}), frozenset({'36', '90'}), frozenset({'36', '34'}), frozenset({'59', '85'}), frozenset({'85', '86'}), frozenset({'39', '86'}), frozenset({'90', '39'}), frozenset({'90', '86'}), frozenset({'36', '85', '34'}), frozenset({'63', '85'}), frozenset({'36', '90', '85'}), frozenset({'90', '34', '85'}), frozenset({'59', '85', '34'}), frozenset({'85', '34', '86'}), frozenset({'34', '39', '86'}), frozenset({'36', '85', '86'}), frozenset({'36', '90', '34'}), frozenset({'59', '86'}), frozenset({'85', 

(5) Implement the Dynamic Itemset Counting Algorithm for Frequent Itemset Generation.

In [ ]:
import itertools

# Dynamic Itemset Counting Algorithm (DIC)
def dic_algorithm(dataset, min_support):
    transactions = len(dataset)
    item_count = defaultdict(int)
    candidate_itemsets = set()

    # Phase 1: Initialize candidate 1-itemsets
    for transaction in dataset:
        for item in transaction:
            item_count[frozenset([item])] += 1
            candidate_itemsets.add(frozenset([item]))

    frequent_itemsets = {item for item in candidate_itemsets if item_count[item] >= min_support}

    # Phase 2: Extend itemsets dynamically
    k = 2
    while candidate_itemsets:
        next_candidates = set()
        for itemset1, itemset2 in itertools.combinations(frequent_itemsets, 2):
            new_itemset = itemset1 | itemset2
            if len(new_itemset) == k:
                next_candidates.add(new_itemset)

        item_count.update({itemset: 0 for itemset in next_candidates})

        for transaction in dataset:
            for itemset in next_candidates:
                if itemset.issubset(transaction):
                    item_count[itemset] += 1

        candidate_itemsets = {itemset for itemset in next_candidates if item_count[itemset] >= min_support}
        frequent_itemsets.update(candidate_itemsets)
        k += 1

    return frequent_itemsets

# Run Dynamic Itemset Counting Algorithm
dic_frequent_patterns = dic_algorithm(dataset, min_support)

print(f"Total number of frequent items (DIC): {len(dic_frequent_patterns)}")
print("Frequent items:", dic_frequent_patterns)

Total number of frequent items (DIC): 51
Frequent items: {frozenset({'85', '90', '86'}), frozenset({'85', '39', '86'}), frozenset({'34', '85', '39', '86'}), frozenset({'85'}), frozenset({'39'}), frozenset({'36'}), frozenset({'34'}), frozenset({'36', '85', '34', '86'}), frozenset({'59', '85', '86'}), frozenset({'85', '90'}), frozenset({'36', '34', '86'}), frozenset({'36', '90', '86'}), frozenset({'90', '34', '86'}), frozenset({'90', '34'}), frozenset({'59', '34', '86'}), frozenset({'85', '39', '34'}), frozenset({'36', '90'}), frozenset({'36', '34'}), frozenset({'59', '85'}), frozenset({'85', '86'}), frozenset({'39', '86'}), frozenset({'90', '39'}), frozenset({'90', '86'}), frozenset({'36', '85', '34'}), frozenset({'63', '85'}), frozenset({'36', '85', '90'}), frozenset({'34', '85', '90'}), frozenset({'59', '85', '34'}), frozenset({'85', '34', '86'}), frozenset({'34', '39', '86'}), frozenset({'36', '85', '86'}), frozenset({'36', '90', '34'}), frozenset({'59', '86'}), frozenset({'85', '39'